# Log All Experimental Results

In [148]:
import csv, os, sys, re, string, json, glob, shutil, random, datetime, math

from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np
import pprint

from IPython.display import HTML as html_print
from IPython.display import Markdown
from IPython.display import clear_output

# from IPython.display import Image
from PIL import Image, ImageDraw, ImageFont
from ipywidgets import Video

import torch
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set()
from matplotlib import pyplot as plt
import matplotlib.patches as patches
%pylab inline

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Use our own built-in customized COCO API.
sys.path.insert(0, "../cocoapi")

In [3]:
# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval
from PythonAPI.pycocotools.coco import COCO
from PythonAPI.pycocotools.cocoeval import COCOeval

import inspect
print("COCO API at:     {}".format(inspect.getfile(COCO)))
print("COCOeval API at: {}".format(inspect.getfile(COCOeval)))

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

COCO API at:     /home/telinwu/research/project_jarvis/modeling/../cocoapi/PythonAPI/pycocotools/coco.py
COCOeval API at: /home/telinwu/research/project_jarvis/modeling/../cocoapi/PythonAPI/pycocotools/cocoeval.py


In [4]:
from coco_eval import calc_iou_individual

## COCO-API Results

## Ego4D SCOD

### Datasets

In [5]:
annots_root = "/local1/hu528/ego4d_data/v1/annotations/"

scod_train_file = "fho_scod_train.json"
scod_train_file = os.path.join(annots_root, scod_train_file)

scod_val_file = "fho_scod_val.json"
scod_val_file = os.path.join(annots_root, scod_val_file)

scod_train_clips = json.load(open(scod_train_file))["clips"]
scod_val_clips = json.load(open(scod_val_file))["clips"]

scod_clips = scod_train_clips + scod_val_clips

print("Train SCOD clips: {}".format(len(scod_train_clips)))
print("Val   SCOD clips: {}".format(len(scod_val_clips)))
print("Total SCOD clips: {}".format(len(scod_clips)))

Train SCOD clips: 19071
Val   SCOD clips: 12801
Total SCOD clips: 31872


### Processing Functions

In [6]:
def stats_on_prediction(
    coco_pred_file,
):
    coco_preds = json.load(open(coco_pred_file))
    d = {}
    for pred in coco_preds:
        image_id = pred["image_id"]
        if image_id not in d:
            d[image_id] = 0
        d[image_id] += 1
    arr = []
    for image_id in d:
        arr.append(d[image_id])
    arr = np.asarray(arr)
    avg_preds = np.mean(arr)
    std_preds = np.std(arr)
    print("Mean Pred BBoxes: {:.3f}".format(avg_preds))
    print("Std  Pred BBoxes: {:.3f}".format(std_preds))

In [7]:
def create_coco_gt_pred_dict(
    coco_gt_file,
    coco_pred_file,
    compensate_missing_pred=False,
):
    preds = json.load(open(coco_pred_file))
    gts = json.load(open(coco_gt_file))["annotations"]

    gt_dict = {}
    for gt in gts:
        image_id = gt["image_id"]
        if image_id not in gt_dict:
            gt_dict[image_id] = []
        gt_dict[image_id].append(gt["bbox"])
    print("Num GT Instances: {}".format(len(gt_dict)))

    preds_dict = {}
    for pred in preds:
        image_id = pred["image_id"]
        if image_id not in preds_dict:
            preds_dict[image_id] = []
        preds_dict[image_id].append({
            "bbox": pred["bbox"],
            "score": pred["score"],
            "category_id": pred["category_id"],
            "image_id": pred["image_id"],
        })
    print("Num Pred Instances: {}".format(len(preds_dict)))
    
    assert len(gt_dict) >= len(preds_dict), (
        "More prediction ids than gt ids, sure the correct file(s)?"
    )

    joint_dict = {}
    for image_id in sorted(gt_dict):
        if compensate_missing_pred and image_id not in preds_dict:
            preds_dict[image_id] = [{
                "bbox": [0, 0, 1, 1],
                "score": 1.0,
                "category_id": 1,
                "image_id": image_id,
            }]
        preds_dict[image_id] = sorted(
            preds_dict[image_id],
            key=lambda x: x["score"],
            reverse=True,
        )
        joint_dict[image_id] = {
            "gt_bboxes": gt_dict[image_id],
            "pred_bboxes": preds_dict[image_id],
        }
    
    print("Num Joint Instances: {}".format(len(joint_dict)))
    return joint_dict


def get_ndarray_coco_results(coco_pred_file):
    # [Nx7] where each row contains {imageID,x1,y1,w,h,score,class}
    if type(coco_pred_file) == str:
        preds = json.load(open(coco_pred_file))
    else:
        preds = coco_pred_file
    arrs = []
    for pred in preds:
        image_id = pred["image_id"]
        x1 = pred["bbox"][0]
        y1 = pred["bbox"][1]
        w  = pred["bbox"][2]
        h  = pred["bbox"][3]
        score = pred["score"]
        label = pred["category_id"]
        arr = [image_id, x1, y1, w, h, score, label]
        arrs.append(arr)
    return np.asarray(arrs)

In [8]:
def get_per_frame_type_ndarray_coco_results(
    coco_gt_file,
    coco_pred_file,
    scod_clips,
    forced_frame_types=None,
    verbose=False
):
    if type(coco_pred_file) == str:
        preds = json.load(open(coco_pred_file))
    else:
        preds = coco_pred_file
        
    if type(coco_gt_file) == str:
        gt_data = json.load(open(coco_gt_file))
        gts, anns = gt_data["images"], gt_data["annotations"]
    else:
        gts, anns = coco_gt_file
        
    frame_type_dict = {}
    for scod in scod_clips:
        video_uid = scod["video_uid"]
        for fr in ["pre", "pnr", "post"]:
            frame_type = "{}_frame".format(fr)
            if frame_type not in scod:
                continue
            frame_num = scod[frame_type]["frame_number"]
            key = "{}_{}".format(video_uid, frame_num)
            if key not in frame_type_dict:
                frame_type_dict[key] = []
            if frame_type not in frame_type_dict[key]:
                frame_type_dict[key].append(frame_type)
            pass
        pass

    id_to_frame_mappings = {}
    for gt in gts:
        file_name = gt["file_name"].split(".")[0]
        video_uid, frame_num = file_name.split("/")
        key = "{}_{}".format(video_uid, frame_num)
        if key not in frame_type_dict:
            raise ValueError(
                "Key {} not in scod clips, sure the right file(s)?".format(key)
            )
        image_id = gt["id"]
        if image_id not in id_to_frame_mappings:
            id_to_frame_mappings[image_id] = []
        id_to_frame_mappings[image_id] += frame_type_dict[key]
        
    per_frame_type_anns = {}
    for ann in anns:
        image_id = ann["image_id"]
        frame_types = id_to_frame_mappings[image_id]
        for frame_type in frame_types:
            if frame_type not in per_frame_type_anns:
                per_frame_type_anns[frame_type] = []
            per_frame_type_anns[frame_type].append(ann)
        pass
    
    per_frame_type_preds = {}
    for pred in preds:
        image_id = pred["image_id"]
        frame_types = id_to_frame_mappings[image_id]
        for frame_type in frame_types:
            if frame_type not in per_frame_type_preds:
                per_frame_type_preds[frame_type] = []
            per_frame_type_preds[frame_type].append(pred)
        pass
    
    per_frame_type_pred_ndarrays = {}
    for fr in ["pre", "pnr", "post"]:
        frame_type = "{}_frame".format(fr)
        if forced_frame_types is not None and fr not in forced_frame_types:
            continue
        per_frame_type_pred_ndarrays[frame_type] = get_ndarray_coco_results(
            per_frame_type_preds[frame_type]
        )
        if verbose:
            print("{}'s cnt = {}".format(frame_type, len(per_frame_type_preds[frame_type])))
    
    return per_frame_type_pred_ndarrays, per_frame_type_anns

In [54]:
def check_gt_pred_match_on_file(
    coco_gt_file,
    coco_pred_file,
):
    print("Checking GT and Pred files matching...")
    if type(coco_gt_file) is str:
        coco_gt_data = json.load(open(coco_gt_file))
    else:
        coco_gt_data = coco_gt_file
    if type(coco_pred_file) is str:
        coco_pred_data = json.load(open(coco_pred_file))
    elif type(coco_pred_file) == np.ndarray:
        _coco = COCO(verbose=False)
        coco_pred_data = _coco.loadNumpyAnnotations(coco_pred_file)
    else:
        coco_pred_data = coco_pred_file
    gt_image_ids = {x["id"]: True for x in coco_gt_data["images"]}
    pred_image_ids = [x["image_id"] for x in coco_pred_data]
    pred_image_ids = list(set(pred_image_ids))
    for _id in pred_image_ids:
        if _id not in gt_image_ids:
            raise ValueError(
                "\nGT file: {}\nPred file: {}\nThey do not match!?".format(coco_gt_file, coco_pred_file))
        pass
    print("Checking complete!")
    pass  # Passed.


def naive_ego4d_scod_coco_results(
    coco_gt_file,
    coco_pred_file,
    original_coco_gt_file=None,
    top_k=None,
    ignore_summaries=None,
    do_not_summarize=False,
    verbose=False,
):  
    if original_coco_gt_file is not None:
        coco_gt = COCO(annotation_file=original_coco_gt_file, verbose=verbose)
        coco_gt = coco_gt.loadRes(coco_gt_file)
        check_gt_pred_match_on_file(original_coco_gt_file, coco_pred_file)
    else:
        coco_gt = COCO(annotation_file=coco_gt_file, verbose=verbose)
        check_gt_pred_match_on_file(coco_gt_file, coco_pred_file)
    coco_dt = coco_gt.loadRes(coco_pred_file)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.params.verbose = verbose
    if top_k is not None:
        coco_eval.params.maxDets = [top_k, top_k, top_k]
    coco_eval.evaluate()
    coco_eval.accumulate()
    if not do_not_summarize:
        coco_eval.summarize(ignore_summaries=ignore_summaries)
    return coco_eval


def comprehensive_ego4d_scod_coco_results(
    coco_gt_file,
    coco_pred_file,
    scod_clips,
    top_k=None,
    forced_frame_types=None,
    ignore_summaries=None,
    return_eval_dicts=False,
    separate_classes=False,
    verbose=False,
):
    object_type_dict = {}
    object_classes = json.load(open(coco_gt_file))["categories"]
    for object_class in object_classes:
        object_id = int(object_class["id"])
        object_name = object_class["name"]
        object_type_dict[object_id] = object_name
    
    per_frame_type_pred_ndarrays, per_frame_type_anns = get_per_frame_type_ndarray_coco_results(
        coco_gt_file=coco_gt_file,
        coco_pred_file=coco_pred_file,
        scod_clips=scod_clips,
        forced_frame_types=forced_frame_types,
        verbose=verbose,
    )
    
    all_coco_evals = {}
    all_coco_gts = {}
    
    if return_eval_dicts:
        original_coco_gt = json.load(open(coco_gt_file))
    
    for fr in ["pre", "pnr", "post"]:
        frame_type = "{}_frame".format(fr)
        if frame_type not in per_frame_type_pred_ndarrays:
            continue
        if fr != "pre":
            print()
        print("---------- {} ----------".format(frame_type))
        curr_frame_type_anns = per_frame_type_anns[frame_type]
        curr_frame_type_pred_ndarrays = per_frame_type_pred_ndarrays[frame_type]

        if separate_classes:
            for object_id in sorted(object_type_dict):
                object_id_indices = np.where(curr_frame_type_pred_ndarrays[:, -1]==object_id)
                object_id_frame_type_pred_ndarrays = curr_frame_type_pred_ndarrays[object_id_indices]
                object_id_frame_type_anns = [ann for ann in curr_frame_type_anns if ann["category_id"] == object_id]
                print("----- {} -----".format(object_type_dict[object_id]))
                _ = naive_ego4d_scod_coco_results(
                    coco_gt_file=object_id_frame_type_anns,
                    coco_pred_file=object_id_frame_type_pred_ndarrays,
                    original_coco_gt_file=coco_gt_file,
                    top_k=top_k,
                    ignore_summaries=ignore_summaries,
                    verbose=verbose,
                )
            # Simply for the eval dict.
            if return_eval_dicts:
                curr_coco_eval = naive_ego4d_scod_coco_results(
                    coco_gt_file=curr_frame_type_anns,
                    coco_pred_file=curr_frame_type_pred_ndarrays,
                    original_coco_gt_file=coco_gt_file,
                    top_k=top_k,
                    ignore_summaries=ignore_summaries,
                    do_not_summarize=True,
                    verbose=verbose,
                )
        else:
            curr_coco_eval = naive_ego4d_scod_coco_results(
                coco_gt_file=curr_frame_type_anns,
                coco_pred_file=curr_frame_type_pred_ndarrays,
                original_coco_gt_file=coco_gt_file,
                top_k=top_k,
                ignore_summaries=ignore_summaries,
                verbose=verbose,
            )

        if return_eval_dicts:
            fr_image_ids = [x["image_id"] for x in curr_frame_type_anns]
            fr_anns = curr_frame_type_anns
            fr_images = [x for x in original_coco_gt["images"] if x["id"] in fr_image_ids]
            fr_coco_gt = {
                "info": original_coco_gt["info"],
                "licenses": original_coco_gt["licenses"],
                "categories": original_coco_gt["categories"],
                "images": fr_images,
                "annotations": fr_anns,
            }
            all_coco_gts[frame_type] = fr_coco_gt
            curr_coco_eval.params.imgIds = fr_image_ids
            curr_coco_eval._paramsEval.imgIds = fr_image_ids
            all_coco_evals[frame_type] = curr_coco_eval

    if return_eval_dicts:
        return all_coco_evals, all_coco_gts
    pass

In [10]:
def get_best_k_scod_preds(
    coco_gt_pred_dict,
    best_k=3,
):
    best_k_data = []
    for image_id in coco_gt_pred_dict:
        coco_gt_pred = coco_gt_pred_dict[image_id]
        gt_bboxes = coco_gt_pred["gt_bboxes"]
        pred_bboxes_scores = coco_gt_pred["pred_bboxes"]
        pred_bboxes_scores = pred_bboxes_scores[:best_k]
        pred_bboxes = [p["bbox"] for p in pred_bboxes_scores]
        best_k_bboxes_to_retain = []
        for gt_bbox in gt_bboxes:
            # xywh -> xyxy.
            _gt_bbox = [
                gt_bbox[0],            gt_bbox[1],
                gt_bbox[0]+gt_bbox[2], gt_bbox[1]+gt_bbox[3],
            ]
            curr_ious = []
            for pred_bbox in pred_bboxes:
                # xywh -> xyxy.
                _pred_bbox = [
                    pred_bbox[0],              pred_bbox[1],
                    pred_bbox[0]+pred_bbox[2], pred_bbox[1]+pred_bbox[3],
                ]
                _iou = calc_iou_individual(_pred_bbox, _gt_bbox)
                curr_ious.append(_iou)
            if len(pred_bboxes) <= 0:
                continue
            curr_ious = np.asarray(curr_ious)
            iou_arg_sort = np.argsort(-curr_ious)
            # Best one out of k.
            best_idx = iou_arg_sort[0]
            best_k_bboxes_to_retain.append(pred_bboxes_scores[best_idx])
            # If used, skip.
            pred_bboxes.pop(best_idx)
        best_k_data += best_k_bboxes_to_retain
    print("Num Best-{} instances: {}".format(best_k, len(best_k_data)))
    return best_k_data

### Single File Exemplar Results

* Model: DETR (Meta AI)
* Training frames = `pnr frames`;  Eval frames = `pnr frames`

In [36]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations/val.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/DETR/ego4dv1_pnr_objects/inference/coco_instances_results.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=8.13s).
Accumulating evaluation results...
DONE (t=2.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.270
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.291
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.298


In [38]:
forced_frame_types = ["pnr"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=8.24s).
Accumulating evaluation results...
DONE (t=1.95s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.270
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.291
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.299


## Main Results

#### SCOD Results
* Model: VideoIntern (Uniformer-L?)
* Training frames = `pnr frames`;  Eval frames = `pnr frames`

In [195]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations/val.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/VideoIntern/ego4dv1_pnr_objects/inference/test.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=38.45s).
Accumulating evaluation results...
DONE (t=8.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.435
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.242
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.534
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.644


In [196]:
stats_on_prediction(
    coco_pred_file,
)

Mean Pred BBoxes: 100.000
Std  Pred BBoxes: 0.000


In [53]:
ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=5.85s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.347
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.194
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.318



* Training frames = `pnr frames`;  Eval frames = `all frames`

In [50]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/VideoIntern/ego4dv2_pre_pnr_post_objects/inference/coco_instances_results_pnr2prepost.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=42.28s).
Accumulating evaluation results...
DONE (t=39.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.215
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.290
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.515
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.638
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=42.93s).
Accumulating evaluation results...
DONE (t=34.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.434
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.241
 Average Recal

In [51]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=5.29s).
Accumulating evaluation results...
DONE (t=0.59s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.297
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.168
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.290
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.290
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.290
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.95s).
Accumulating evaluation results...
DONE (t=0.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.347
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.196
 Average Recall 

* VideoIntern-`Best`

In [191]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions"
    "/scod/OUTPUT_RESULTS/Swin-L-IN-22K+O365/pre_post_pnr_results.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=39.14s).
Accumulating evaluation results...
DONE (t=8.75s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.492
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.340
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.662
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.737
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=39.18s).
Accumulating evaluation results...
DONE (t=9.53s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.570
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.386
 Average Recall 

* VideoIntern-`Best` with **batch-size = 8**

In [15]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/VideoIntern"
    "/ego4dv2_pre_pnr_post_objects_bz8/inference"
    "/pre_post_pnr_results_epoch_5_narrated_gt_srl_arg1.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=40.46s).
Accumulating evaluation results...
DONE (t=8.59s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.009
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.031
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.129
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=41.73s).
Accumulating evaluation results...
DONE (t=8.42s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

#### SCOD Results
* Model: GLIP
* Training frames = `pnr frames`;  Eval frames = `pnr frames`
* Train: (Naive) detection with `object_of_change` as prompts
* Eval:  (Naive) detection with `object_of_change` as prompts

In [62]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations/val.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/naive_scod_finetuning_v1/eval_ego4d_scod_val_finetuned_45K"
    "/eval/model_0045000/inference/test/bbox.json"
)

forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=11.03s).
Accumulating evaluation results...
DONE (t=2.50s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.248
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.430
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.248
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.325
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.580


In [63]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=3.17s).
Accumulating evaluation results...
DONE (t=0.55s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.326
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.198
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.325
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.325
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.325


#### SCOD Results
* Model: GLIP-**L**
* Training frames = `pnr frames`;  Eval frames = `all frames`
* Train: (Naive) detection with `object_of_change` as prompts
* Eval:  (Naive) detection with `object_of_change` as prompts

In [128]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/naive_scod_finetuning_large_v1/eval_ego4d_scod_val_finetuned_45K_pnr2prepost"
    "/eval/model_0045000/inference/test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=7.70s).
Accumulating evaluation results...
DONE (t=2.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.268
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.563
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.585
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=8.02s).
Accumulating evaluation results...
DONE (t=1.78s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.313
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxD

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: (Naive) detection with `object_of_change` as prompts
* Eval:  (Naive) detection with `object_of_change` as prompts

In [127]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val.json"
)
coco_pred_file = (    
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/naive_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K"
    "/eval/model_0045000/inference/test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=14.71s).
Accumulating evaluation results...
DONE (t=3.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.269
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.428
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.586
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.632
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=14.94s).
Accumulating evaluation results...
DONE (t=3.32s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.297
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

#### SCOD Results
* Model: GLIP
* Training frames = `pnr frames`;  Eval frames = `pnr frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `GT-SRL-ARG1`

In [54]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v1/eval_ego4d_narrated_scod_val_finetuned_45K_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=8.46s).
Accumulating evaluation results...
DONE (t=1.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.275
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.462
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.273
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.511
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.533


In [55]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Evaluate annotation type *bbox*
DONE (t=4.83s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.382
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.347


* Training frames = `pnr frames`;  Eval frames = `all frames`

In [56]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v1/eval_ego4d_narrated_scod_val_finetuned_45K_pnr2prepost_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=8.25s).
Accumulating evaluation results...
DONE (t=1.85s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.257
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.427
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.257
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.528
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=10.42s).
Accumulating evaluation results...
DONE (t=1.80s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.273
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.458
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.270
 Average Recall  

In [57]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.84s).
Accumulating evaluation results...
DONE (t=0.56s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.350
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.222
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.337
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.02s).
Accumulating evaluation results...
DONE (t=0.54s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.380
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.233
 Average Recall 

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `SRL-ARG1-only`
* Eval:  Grounding with `SRL-ARG1-only`

In [63]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_srl_arg1_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_arg1_only"
    "/eval_ego4d_narrated_scod_val_finetuned_45K_arg1_only/eval/model_0045000/inference"
    "/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

eval_dicts, coco_gts = comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    return_eval_dicts=True,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=12.50s).
Accumulating evaluation results...
DONE (t=2.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.579
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.407
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.443
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.645
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.681
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=12.49s).
Accumulating evaluation results...
DONE (t=2.72s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.604
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.416
 Average Recall 

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `GT-SRL-ARG1`

In [64]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2/eval_ego4d_narrated_scod_val_finetuned_45K_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

eval_dicts, coco_gts = comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    return_eval_dicts=True,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=11.92s).
Accumulating evaluation results...
DONE (t=2.64s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.483
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.568
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.607
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=11.78s).
Accumulating evaluation results...
DONE (t=2.56s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.504
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.327
 Average Recall 

In [13]:
coco_gt_pred_dict = create_coco_gt_pred_dict(
    coco_gt_file,
    coco_pred_file,
    compensate_missing_pred=True,
)

Num GT Instances: 36885
Num Pred Instances: 36283
Num Joint Instances: 36885


In [14]:
best_k_data = get_best_k_scod_preds(
    coco_gt_pred_dict,
    best_k=4,
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=best_k_data,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Num Best-4 instances: 37252
---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.97s).
Accumulating evaluation results...
DONE (t=0.88s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.597
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.399
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.513
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.515
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.515
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.62s).
Accumulating evaluation results...
DONE (t=4.83s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.622
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 

In [15]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.06s).
Accumulating evaluation results...
DONE (t=0.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.404
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.272
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.95s).
Accumulating evaluation results...
DONE (t=0.72s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.422
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.285
 Average Recall 

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `SRL-ARG1-only`

In [26]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_srl_arg1_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2/eval_ego4d_narrated_scod_val_finetuned_45K_arg1_only"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=9.84s).
Accumulating evaluation results...
DONE (t=2.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.237
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.240
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.563
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.597
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=9.73s).
Accumulating evaluation results...
DONE (t=2.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.249
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.250
 Average Recall   

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `Full-Sentence`

In [27]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_ground_full_sentence.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2/eval_ego4d_narrated_scod_val_finetuned_45K_full_sent"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.72s).
Accumulating evaluation results...
DONE (t=3.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.119
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.364
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.470
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.479
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.77s).
Accumulating evaluation results...
DONE (t=1.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.220
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.129
 Average Recall   

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `Full-Sentence`
* Eval:  Grounding with `Full-Sentence`

In [28]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_ground_full_sentence.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2_full_sent/eval_ego4d_narrated_scod_val_finetuned_45K_full_sent"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=18.75s).
Accumulating evaluation results...
DONE (t=3.57s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.293
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.480
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.298
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.350
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.554
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.606
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=15.76s).
Accumulating evaluation results...
DONE (t=3.47s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.299
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.499
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.300
 Average Recall 

In [33]:
coco_gt_pred_dict = create_coco_gt_pred_dict(
    coco_gt_file,
    coco_pred_file,
    compensate_missing_pred=True,
)

Num GT Instances: 36885
Num Pred Instances: 36885
Num Joint Instances: 36885


In [34]:
best_k_data = get_best_k_scod_preds(
    coco_gt_pred_dict,
    best_k=4,
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=best_k_data,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Num Best-4 instances: 37275
---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=5.67s).
Accumulating evaluation results...
DONE (t=0.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.634
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.404
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.494
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=2.82s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.653
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 

In [35]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.73s).
Accumulating evaluation results...
DONE (t=0.60s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.381
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.249
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.350
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.350
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.350
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.44s).
Accumulating evaluation results...
DONE (t=0.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.399
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.253
 Average Recall 

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `SRL-ARG1-only`
* Eval:  Grounding with `SRL-ARG1-only`

In [203]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_srl_arg1_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2_arg1_only/eval_ego4d_narrated_scod_val_finetuned_45K_arg1_only"
    "/eval/model_0035000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=18.34s).
Accumulating evaluation results...
DONE (t=3.84s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.288
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.474
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.296
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.558
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.619
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=25.52s).
Accumulating evaluation results...
DONE (t=4.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.507
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.311
 Average Recall 

In [204]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.86s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.372
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.247
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.348
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.83s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.404
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.260
 Average Recall 

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `Random-GT-Word`
* Eval:  Grounding with `Random-GT-Word`

In [199]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_random_word.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2_rand_word/eval_ego4d_narrated_scod_val_finetuned_45K_rand_word"
    "/eval/model_0040000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=7.90s).
Accumulating evaluation results...
DONE (t=1.85s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.259
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.422
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.262
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.401
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.596
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.614
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=13.54s).
Accumulating evaluation results...
DONE (t=1.90s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.268
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.442
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.268
 Average Recall  

In [200]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=2.98s).
Accumulating evaluation results...
DONE (t=0.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.367
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.237
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.401
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.401
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.401
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=8.41s).
Accumulating evaluation results...
DONE (t=0.60s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.394
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.243
 Average Recall 

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `pnr frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `GT-SRL-ARG1`

In [59]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v1/eval_ego4d_narrated_scod_val_finetuned_45K_pnr2prepost_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=9.71s).
Accumulating evaluation results...
DONE (t=2.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.348
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.529
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.360
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.591
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.613
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=9.00s).
Accumulating evaluation results...
DONE (t=2.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.377
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.388
 Average Recall   

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `V-GT-SRL-ARG1`
* Eval:  Grounding with `V-GT-SRL-ARG1`

In [126]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_v_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_v_gt_arg1/eval_ego4d_narrated_scod_val_finetuned_45K_v_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.63s).
Accumulating evaluation results...
DONE (t=3.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.399
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.447
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.647
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.676
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.55s).
Accumulating evaluation results...
DONE (t=3.07s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

`no_hand`

In [69]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_v_gt_srl_arg1_no_hand.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP"
    "/narrated_scod_finetuning_large_v2_v_gt_srl_arg1_no_hand/eval_ego4d_narrated_scod_val_finetuned_45K_no_hand"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=11.88s).
Accumulating evaluation results...
DONE (t=2.68s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.591
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.412
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.439
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.641
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.675
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.42s).
Accumulating evaluation results...
DONE (t=2.78s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1`
* Eval:  Grounding with `GT-SRL-ARG1`

In [58]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2/eval_ego4d_narrated_scod_val_finetuned_45K_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=13.43s).
Accumulating evaluation results...
DONE (t=3.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.386
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.575
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.403
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.633
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.666
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=13.40s).
Accumulating evaluation results...
DONE (t=2.97s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.397
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.595
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.408
 Average Recall 

In [206]:
coco_gt_pred_dict = create_coco_gt_pred_dict(
    coco_gt_file,
    coco_pred_file,
    compensate_missing_pred=True,
)

Num GT Instances: 36885
Num Pred Instances: 36609
Num Joint Instances: 36885


In [207]:
best_k_data = get_best_k_scod_preds(
    coco_gt_pred_dict,
    best_k=4,
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=best_k_data,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Num Best-4 instances: 37266
---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=2.80s).
Accumulating evaluation results...
DONE (t=0.67s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.433
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.675
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.455
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.546
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.549
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.549
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=2.86s).
Accumulating evaluation results...
DONE (t=0.62s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.436
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.689
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 

`no_hand`

In [70]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1_no_hand.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP"
    "/narrated_scod_finetuning_large_v2_gt_srl_arg1_no_hand/eval_ego4d_narrated_scod_val_finetuned_45K_no_hand"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.31s).
Accumulating evaluation results...
DONE (t=2.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.566
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.387
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.430
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.624
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.660
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.40s).
Accumulating evaluation results...
DONE (t=13.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50      | area=   all | m

#### SCOD Results
* Model: GLIP
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `first word`
* Eval:  Grounding with `first word`

In [216]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_first_word.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2_first_word/eval_ego4d_narrated_scod_val_finetuned_45K_first_word"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=20.86s).
Accumulating evaluation results...
DONE (t=4.28s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.301
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.489
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.304
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.564
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.627
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=20.07s).
Accumulating evaluation results...
DONE (t=4.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.504
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.311
 Average Recall 

In [217]:
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    top_k=1,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=4.24s).
Accumulating evaluation results...
DONE (t=0.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.386
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.252
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.354
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=3.66s).
Accumulating evaluation results...
DONE (t=0.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.402
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.259
 Average Recall 

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `zero-shot`;  Eval frames = `all frames`
* Train: `N/A`
* Eval:  Grounding with `GT-SRL-ARG1`

In [212]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2/eval_ego4d_narrated_scod_val_zero_shot"
    "/eval/glip_large_model/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Evaluate annotation type *bbox*
DONE (t=8.90s).
Accumulating evaluation results...
DONE (t=2.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.330
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.292
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.466
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.499
---------- pnr_frame ----------
Evaluate annotation type *bbox*
DONE (t=9.18s).
Accumulating evaluation results...
DONE (t=2.10s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.323
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.194
 Average Recall   

### With `GPT`

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GPT-v1-W/o-Tool` with `tool`s **NOT** it
* Eval:  Grounding with `GPT-v1-W/o-Tool` with `tool`s **NOT** it

In [127]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/all_frames/narrated"
    "/gpt_v1_wo_tool/val_scod_all_frames_narrated_gpt_v1_wo_tool.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
    "/gpt_v1_wo_tool/ego4d_scod_all_frames_narrated_gpt_v1_wo_tool"
    "/eval_at_50K/eval/model_0050000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=13.40s).
Accumulating evaluation results...
DONE (t=30.53s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.361
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.545
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.375
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.425
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.663

---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=19.55s).
Accumulating evaluation results...
DONE (t=4.80s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50      | area=   all | 

### On `Tools`

#### SCOD Results
* Model: GLIP-**L**
* Training frames = `all frames`;  Eval frames = `all frames`
* Train: Grounding with `GT-SRL-ARG1` with `tool`s in it
* Eval:  Grounding with `GT-SRL-ARG1` with `tool`s in it, **only eval on tools!**

In [23]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_with_tool_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool"
    "/eval_ego4d_narrated_scod_val_finetuned_65K_with_tool_only"
    "/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.31s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.466
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.710
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.514
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.637
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.653
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.91s).
Accumulating evaluation results...
DONE (t=0.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.431
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxD

On `object of change`

In [ ]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool"
    "/eval_ego4d_narrated_scod_val_finetuned_65K_gt_arg1_with_tool"
    "/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

## Separating `Tool` and `OOC`

In [57]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    "/first1000_narrated_gt_srl_arg1_with_tool_strict.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool_new0527"
    "/testing/eval/model_0008000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    separate_classes=True,
    verbose=False,
)

---------- pre_frame ----------
----- object_of_change -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.439
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.468
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.636
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.652
----- tool -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.22s).
Accumulating evaluation results...
DONE (t=0.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50      | area

In [58]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    "/first1000_narrated_gt_srl_arg1_with_tool_strict.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool_new0527"
    "/testing/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    separate_classes=True,
    verbose=False,
)

---------- pre_frame ----------
----- object_of_change -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.576
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.456
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.655
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.672
----- tool -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.469
 Average Precision  (AP) @[ IoU=0.50      | area

In [64]:
coco_gt_file = (
    "/local1/bryanzhou008/jarvis/GPT/val_res.json"
)
coco_pred_file = (
    # "/home/telinwu/research/GLIP/test_gpt/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
    "/gpt_v1_with_tool/ego4d_scod_all_frames_narrated_gpt_v1_with_tool/eval_at_45K"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    separate_classes=True,
    verbose=False,
)

---------- pre_frame ----------
----- object_of_change -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=19.72s).
Accumulating evaluation results...
DONE (t=4.25s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.512
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.336
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.393
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.591
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.643
----- tool -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=1.31s).
Accumulating evaluation results...
DONE (t=0.39s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.228
 Average Precision  (AP) @[ IoU=0.50      | are

In [65]:
coco_gt_file = (
    "/local1/bryanzhou008/jarvis/GPT/val_res.json"
)
coco_pred_file = (
    "/home/telinwu/research/GLIP/test_gpt/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    separate_classes=True,
    verbose=False,
)

---------- pre_frame ----------
----- object_of_change -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=11.30s).
Accumulating evaluation results...
DONE (t=2.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.517
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.357
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.635
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.663
----- tool -----
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=1.05s).
Accumulating evaluation results...
DONE (t=0.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.199
 Average Precision  (AP) @[ IoU=0.50      | are

### Confusion Matrix for Result Comparisons

In [11]:
def get_conf_mats_with_coco_eval_dicts(
    eval_dicts_model1,
    eval_dicts_model2,
    coco_gt_file_joint,
    maxDets=100,
    query_iou_thrs=50,
):
    gt_image_ids = [x["image_id"] for x in json.load(open(coco_gt_file_joint))["annotations"]]
    gt_image_ids = sorted(list(set(gt_image_ids)))

    for fr in ["pre", "pnr", "post"]:
        frame_type = "{}_frame".format(fr)
        if frame_type not in eval_dicts_model1 or frame_type not in eval_dicts_model2:
            continue
        perf_dict_model1 = construct_performance_dict(
            eval_dicts_model1[frame_type],
            maxDets=maxDets,
            query_iou_thrs=query_iou_thrs,
        )
        perf_dict_model2 = construct_performance_dict(
            eval_dicts_model2[frame_type],
            maxDets=maxDets,
            query_iou_thrs=query_iou_thrs,
        )

        conf_mat = {
            "11": 0,
            "10": 0,
            "01": 0,
            "00": 0,
        }
        id_visited = {}

        print("========== {}-frame ==========".format(fr))
        for _id in perf_dict_model1:
            if _id in id_visited:
                continue
            res1 = perf_dict_model1[_id]
            if _id not in perf_dict_model2:
                res2 = 0
            else:
                res2 = perf_dict_model2[_id]
            res_str = "{}{}".format(res1, res2)
            assert res_str in conf_mat
            conf_mat[res_str] += 1
            id_visited[_id] = True
        for _id in perf_dict_model2:
            if _id in id_visited:
                continue
            res2 = perf_dict_model2[_id]
            if _id not in perf_dict_model1:
                res1 = 0
            else:
                res1 = perf_dict_model1[_id]
            res_str = "{}{}".format(res1, res2)
            assert res_str in conf_mat
            conf_mat[res_str] += 1
            id_visited[_id] = True
        lst = [
            ["Correct", conf_mat["11"], conf_mat["10"]],
            ["Wrong",   conf_mat["01"], conf_mat["00"]],
        ]
        df = pd.DataFrame(lst, columns =["Model(1/2)", "Correct", "Wrong"])
        print(df)
    pass
    ##########

In [15]:
coco_gt_file_joint = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file_model1 = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions"
    "/scod/pre_post_pnr_results_epoch_8_narrated_gt_srl_arg1.json"
)
coco_pred_file_model2 = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2/eval_ego4d_narrated_scod_val_finetuned_45K_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)

forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

eval_dicts_model1, coco_gts_1 = comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file_joint,
    coco_pred_file=coco_pred_file_model1,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    return_eval_dicts=True,
    verbose=False,
)

eval_dicts_model2, coco_gts_2 = comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file_joint,
    coco_pred_file=coco_pred_file_model2,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    return_eval_dicts=True,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=39.26s).
Accumulating evaluation results...
DONE (t=8.24s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.316
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.485
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.326
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.388
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.628
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.704
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=40.30s).
Accumulating evaluation results...
DONE (t=8.22s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

In [16]:
get_conf_mats_with_coco_eval_dicts(
    eval_dicts_model1,
    eval_dicts_model2,
    coco_gt_file_joint,
    maxDets=100,
    query_iou_thrs=50,
)

========== pre-frame ==========
  Model(1/2)  Correct  Wrong
0    Correct    10525   1216
1      Wrong      379    521
========== pnr-frame ==========
  Model(1/2)  Correct  Wrong
0    Correct    10915   1217
1      Wrong      354    470
========== post-frame ==========
  Model(1/2)  Correct  Wrong
0    Correct     9764   1283
1      Wrong      382    566


## Results To Inspect

In [12]:
def construct_performance_dict(
    coco_eval,
    maxDets=None,
    query_iou_thrs=50,
):
    perf_dict = {}
    p = coco_eval.params
    _pe = coco_eval._paramsEval
    I0 = len(_pe.imgIds)
    A0 = len(_pe.areaRng)
    T = p.iouThrs
    area_all_idx = 0
    ooc_cat_idx = p.catIds.index(1)

    query_row = ooc_cat_idx*A0*I0
    query_col = area_all_idx*I0

    setI = set(_pe.imgIds)
    setI = sorted(list(setI))
    i_list = [i for n, i in enumerate(p.imgIds) if i in setI]

    if maxDets is not None:
        maxDet = maxDets
    else:
        maxDet = p.maxDets[-1]
    E = [coco_eval.evalImgs[query_row + query_col + i] for i in i_list]

    query_iou_thrs = query_iou_thrs/100
    query_iou_idx = T.tolist().index(query_iou_thrs)

    iou_per_img_perfs = [e['dtMatches'][query_iou_idx,0:maxDet] for e in E]
    
    for idx in range(len(i_list)):
        iou_img_perf = iou_per_img_perfs[idx]
        if_detected = sum([1 if x > 0 else 0 for x in iou_img_perf]) != 0
        if_detected = int(if_detected)
        perf_dict[i_list[idx]] = if_detected
    
    return perf_dict


def analyze_perf_dict(
    perf_dict,
    criteria=None,
):
    if criteria is None:
        return None
    crit_sub_dict = {}
    fulfill_cnt = 0
    not_fulfill_cnt = 0
    for idx in perf_dict:
        if criteria(perf_dict[idx]):
            crit_sub_dict[idx] = perf_dict[idx]
            fulfill_cnt += 1
        else:
            not_fulfill_cnt += 1
    print("Fulfill Criteria = {} / {}".format(fulfill_cnt, len(perf_dict)))
    return crit_sub_dict


def get_criteria_sub_dict(
    crit_sub_dict,
    coco_gt_file=None,
    coco_gt_file_name=None,
    select_criteria=None,
    suffix=None,
):
    crit_image_ids = sorted(list(crit_sub_dict.keys()))
    interested_ids = select_criteria(crit_image_ids)
    
    if type(coco_gt_file) is str:
        coco_gt = json.load(open(coco_gt_file))
        coco_gt_file_name = coco_gt_file
    else:
        coco_gt = coco_gt_file
        assert coco_gt_file_name is not None
    new_gt_dict = {}
    new_gt_dict["info"] = coco_gt["info"]
    new_gt_dict["licenses"] = coco_gt["licenses"]
    new_gt_dict["categories"] = coco_gt["categories"]
    new_gt_dict["images"] = [x for x in coco_gt["images"] if x["id"] in interested_ids]
    new_gt_dict["annotations"] = [x for x in coco_gt["annotations"] if x["image_id"] in interested_ids]
    refined_file = coco_gt_file_name.replace(".json", "_{}.json".format(suffix))
    json.dump(
        new_gt_dict,
        open(refined_file, "w"),
        indent=4
    )
    print("Saving refined file (cnt={}) to: {}".format(len(new_gt_dict), refined_file))
    return None

In [25]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations_all_frames/val_narrated_gt_srl_arg1.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    "/GLIP/narrated_scod_finetuning_v2/eval_ego4d_narrated_scod_val_finetuned_45K_gt_arg1"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
forced_frame_types = ["pre", "pnr", "post"]
ignore_summaries = {
    "area": ["small", "medium", "large"],
}

eval_dicts, coco_gts = comprehensive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    scod_clips=scod_val_clips,
    forced_frame_types=forced_frame_types,
    ignore_summaries=ignore_summaries,
    return_eval_dicts=True,
    verbose=False,
)

---------- pre_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.66s).
Accumulating evaluation results...
DONE (t=2.86s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.483
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.568
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.607
---------- pnr_frame ----------
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=12.89s).
Accumulating evaluation results...
DONE (t=2.66s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=   all | ma

In [33]:
frame_type = "pnr_frame"

perf_dict = construct_performance_dict(
    eval_dicts[frame_type],
    maxDets=100,
)

crit_sub_dict = analyze_perf_dict(
    perf_dict,
    criteria=lambda x: x > 0,
)

def select_criteria(x):
    # return [24640] + [17671] + x[-9:]
    return [21703, 31204, 28063]

get_criteria_sub_dict(
    crit_sub_dict,
    coco_gt_file=coco_gts[frame_type],
    coco_gt_file_name=coco_gt_file,
    # coco_gt_file_name=(
    #     "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    #     "/val_narrated_gt_srl_arg1_with_tool.json"
    # ),
    select_criteria=select_criteria,
    # suffix="first_10_corrects",
    suffix="tiny_for_tools",
)

get_criteria_sub_dict(
    crit_sub_dict,
    coco_gt_file=(
        "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
        "/val_narrated_gt_srl_arg1_with_tool.json"
    ),
    select_criteria=select_criteria,
    suffix="tiny_tool_only",
)

Fulfill Criteria = 11269 / 12956
Saving refined file (cnt=5) to: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val_narrated_gt_srl_arg1_tiny_for_tools.json
Saving refined file (cnt=5) to: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames/val_narrated_gt_srl_arg1_with_tool_tiny_tool_only.json


## Two-File Re-Ranking

* Check the section `Two-File Re-Ranking` in `exploring_multimodal_models.ipynb` if some file **is missing**!

In [74]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    # "/val_narrated_gt_srl_arg1_first_10_corrects.json"
    "/val_narrated_gt_srl_arg1_tiny_for_tools.json" 
)
coco_pred_file_before = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP"
    # "/narrated_scod_finetuning_large_v2/first_10_corrects/eval/model_0045000"
    "/narrated_scod_finetuning_large_v2/tiny_exp_for_tools/eval/model_0045000"
    "/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file_after = (
    "test.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}

print("===== Before Re-Ranking =====")
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file_before,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)
print()
print("===== After  Re-Ranking =====")
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file_after,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

===== Before Re-Ranking =====
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.094
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.218
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.084
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.400
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.400

===== After  Re-Ranking =====
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.147
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets

In [92]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    # "/val_narrated_gt_srl_arg1_first_10_corrects.json"
    "/val_narrated_gt_srl_arg1_with_tool_strict.json" 
)
coco_pred_file_before = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP"
    "/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool"
    "/eval_ego4d_narrated_scod_val_finetuned_65K_with_tool_strict"
    "/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file_after = (
    "test_new.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}

print("===== Before Re-Ranking =====")
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file_before,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)
# print()
# print("===== After  Re-Ranking =====")
# coco_eval = naive_ego4d_scod_coco_results(
#     coco_gt_file=coco_gt_file,
#     coco_pred_file=coco_pred_file_after,
#     top_k=None,
#     ignore_summaries=ignore_summaries,
#     verbose=False,
# )

===== Before Re-Ranking =====
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=14.32s).
Accumulating evaluation results...
DONE (t=0.87s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.094
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.063
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.065
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.106
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.119


In [146]:
print("===== After  Re-Ranking =====")
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file_after,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

===== After  Re-Ranking =====
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=14.79s).
Accumulating evaluation results...
DONE (t=0.96s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.033
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.053
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.059


In [122]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/coco_annotations_all_frames"
    # "/val_narrated_gt_srl_arg1_first_10_corrects.json"
    "/val_narrated_gt_srl_arg1_with_tool_strict.json" 
)
coco_pred_file_before = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP"
    "/narrated_scod_finetuning_large_v2_gt_srl_arg1_with_tool"
    "/eval_ego4d_narrated_scod_val_finetuned_65K_with_tool_strict"
    "/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file_after = (
    "test_new.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}

print("===== Only Tools =====")
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file_before,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

===== Only Tools =====
Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=2.05s).
Accumulating evaluation results...
DONE (t=0.43s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.443
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.691
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.457
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.497
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.636


## Others

In [73]:
# TBD

# Misc.

In [39]:
# TBD

# Ego4D SCOD Official Test-Set

In [183]:
input_official_test_set = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_data/v1/annotations/fho_scod_test_unannotated.json"
)
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    "/coco_annotations/test_narrated_srl_arg1_only_no_hand.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
    # "/GLIP/narrated_scod_finetuning_large_v2/scod_testset/eval/model_0045000"
    "/GLIP/narrated_scod_finetuning_large_v2_arg1_only/scod_testset/eval/model_0050000"
    "/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
    "/gpt_v1_srl_arg1_with_tool"
    "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool"
    "/eval_scod_testset/eval_at_45k/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
    "/gpt_v3_srl_arg1_with_tool"
    "/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr"
    "/eval_scod_testset/eval_at_45k/inference/narrated_ego4d_test/bbox.json"
)

coco_preds = json.load(open(coco_pred_file))
image_id_bbox = {}
for pred in coco_preds:
    image_id = pred["image_id"]
    bbox = pred["bbox"]
    score = pred["score"]
    if image_id not in image_id_bbox:
        image_id_bbox[image_id] = []
    image_id_bbox[image_id].append({
        "x": bbox[0],
        "y": bbox[1],
        "width": bbox[2],
        "height": bbox[3],
        "score": score,
    })
for image_id in image_id_bbox:
    image_id_bbox[image_id] = sorted(
        image_id_bbox[image_id],
        key=lambda x: x["score"],
        reverse=True
    )
coco_gt = json.load(open(coco_gt_file))

input_official_test = json.load(open(input_official_test_set))
clips = input_official_test["clips"]
assert len(clips) == len(coco_gt["images"])
print(len(clips), len(coco_gt["images"]))

image_id = 0
image_id_not_predicted = []
for clip in clips:
    if image_id not in image_id_bbox:
        image_id_not_predicted.append(image_id)
    else:
        clip["pnr_frame"]["bbox"] = image_id_bbox[image_id]
    image_id += 1
    
# output_official_test = {}
# output_official_test["version"] = input_official_test["version"]
# output_official_test["date"] = input_official_test["date"]
# output_official_test["description"] = input_official_test["description"]
# output_official_test["split"] = input_official_test["split"]
# output_official_test["clips"] = clips
output_official_test = clips
json.dump(
    output_official_test,
    open(
        (
            # "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines"
            # "/GLIP/narrated_scod_finetuning_large_v2/scod_testset/eval/model_0045000"
            # "/GLIP/narrated_scod_finetuning_large_v2_arg1_only/scod_testset/eval/model_0050000"
            # "/inference/narrated_ego4d_test/to_submit.json"
            "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
            "/gpt_v3_srl_arg1_with_tool"
            "/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr"
            "/eval_scod_testset/eval_at_45k/inference/narrated_ego4d_test/to_submit.json"
        ),
        "w"
    ),
    indent=2,
)

print("Not predicted image count: {}".format(len(image_id_not_predicted)))

662898 662898
Not predicted image count: 14002


In [134]:
print(len(image_id_bbox), len(image_id_bbox)+1880)
print(image_id_not_predicted[:10])

NameError: name 'image_id_bbox' is not defined

In [133]:
del output_official_test
del coco_preds
del coco_gt
del image_id_bbox
del input_official_test
del clips

## Insert BLIP Captions

In [144]:
raw_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/test_set_coco_formatted_closest_narrations.json"
)

blip_captions_file = (
    "./glip_result_json_files/scod_testset_lavis_captions.json"
)

raw_data = json.load(open(raw_file))
blip_captions = json.load(open(blip_captions_file))

raw_images = raw_data["images"]
for idx in tqdm(range(len(raw_images)), desc="BLIPv2"):
    raw_image = raw_images[idx]
    file_name = raw_image["file_name"]
    del raw_image["closest_narrations"]
    blip_caption = blip_captions[file_name][0]
    raw_image["caption"] = blip_caption
    # Assumes full sentence for now.
    len_cap = len(blip_caption)
    tokens_positive_eval = [ [ [0, len_cap] ] ]
    raw_image["tokens_positive_eval"] = tokens_positive_eval
    raw_data["images"][idx] = raw_image

json.dump(
    raw_data,
    open(raw_file.replace("closest_narrations", "blip_ground_full_sentence"), "w"),
    indent=4,
)

BLIPv2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 662898/662898 [00:01<00:00, 365720.57it/s]


## Chunking for Parallel Test-set Inference

In [165]:
import copy, shutil

def chunk_ego4d_testset(
    input_file_to_chunk=None,
    num_chunks=None,
    glip_code_base_folder=None,
    task_type=None,
    output_scripts_folder_name=None,
    output_scripts_folder_root=None,
    output_configs_folder_root=None,
    output_chunked_file_root=None,
    output_eval_results_folder=None,
    model_checkpoint=None,
    model_config=None,
):
    assert task_type in ["narrated", "od"]
    
    # Templates.
    config_template = os.path.join(
        glip_code_base_folder,
        "configs/paper_results/ego4d_scod/test_set/ego4d_scod_testset_{}_template.yaml".format(task_type)
    )
    assert os.path.exists(config_template)
    config_lines = open(config_template).readlines()
    
    eval_script_template = os.path.join(
        glip_code_base_folder,
        "paper_scripts/ego4d_scod/test_set/ego4d_scod_testset_{}_template.sh".format(task_type)
    )
    assert os.path.exists(eval_script_template)
    eval_script_lines = open(eval_script_template).readlines()
    
    # Output folders.
    output_scripts_folder_root = os.path.join(
        glip_code_base_folder,
        output_scripts_folder_root,
    )
    assert os.path.exists(output_scripts_folder_root)
    output_scripts_folder = os.path.join(
        output_scripts_folder_root,
        output_scripts_folder_name,
    )
    if os.path.exists(output_scripts_folder):
        overwrite = input("The current folder set {} exists, overwrite (y/n)? ".format(output_scripts_folder))
        if overwrite.lower() == "y":
            shutil.rmtree(output_scripts_folder)
        else:
            print("Aborting.")
            return None
            
    if os.path.exists(output_scripts_folder):
        shutil.rmtree(output_scripts_folder)
    os.makedirs(output_scripts_folder)
    # print(output_scripts_folder)
    
    output_configs_folder_name = output_scripts_folder_name
    output_configs_folder_root = os.path.join(
        glip_code_base_folder,
        output_configs_folder_root,
    )
    assert os.path.exists(output_configs_folder_root)
    output_configs_folder = os.path.join(
        output_configs_folder_root,
        output_configs_folder_name,
    )
    if os.path.exists(output_configs_folder):
        shutil.rmtree(output_configs_folder)
    os.makedirs(output_configs_folder)
    # print(output_configs_folder)
    
    output_data_folder = os.path.join(
        output_chunked_file_root,
        output_scripts_folder_name,
    )
    if os.path.exists(output_data_folder):
        shutil.rmtree(output_data_folder)
    os.makedirs(output_data_folder)
    
    # Chunking.
    input_file_name = input_file_to_chunk.split("/")[-1]

    print("-"*50)
    input_data = json.load(open(input_file_to_chunk))
    step_size = len(input_data["images"]) // num_chunks
    step_curr = 0
    per_chunk_images = 0
    for i in range(num_chunks):
        curr_chunk_data = copy.deepcopy(input_data)
        images = curr_chunk_data["images"]
        annots = None
        if "annotations" in curr_chunk_data:
            annots = curr_chunk_data["annotations"] 
        if i == num_chunks - 1:
            curr_chunk_images = images[step_curr:]
        else:
            curr_chunk_images = images[step_curr:step_curr+step_size]
        step_curr = step_curr + step_size
        per_chunk_images += len(curr_chunk_images)
        
        curr_chunk_file_name = input_file_name.replace(".json", "_{}.json".format(i+1))
        curr_chunk_file_path = os.path.join(
            output_data_folder,
            curr_chunk_file_name,
        )
        gpu_id = i % 8
        print("GPU: {}\nFile: {}\nData Points: {}".format(gpu_id, curr_chunk_file_path, len(curr_chunk_images)))
        curr_chunk_image_ids = [x["id"] for x in curr_chunk_images]
        curr_chunk_data["images"] = curr_chunk_images
        if "annotations" in curr_chunk_data and annots is not None:
            curr_chunk_data["annotations"] = [a for a in annots if a["image_id"] in curr_chunk_image_ids]
        json.dump(
            curr_chunk_data,
            open(curr_chunk_file_path, "w"),
            indent=4,
        )
        
        # Configs.
        config_path = os.path.join(
            output_configs_folder,
            "ego4d_scod_testset_{}_{}_{}.yaml".format(task_type, output_scripts_folder_name, i+1)
        )
        config_path_to_write_in_script = config_path.replace("{}/".format(glip_code_base_folder), "")
        config_file = open(config_path, "w")
        for line in config_lines:
            if "TRAIN_JSON_FILE" in line:
                line = line.replace("TRAIN_JSON_FILE", curr_chunk_file_path)
            elif "TEST_JSON_FILE" in line:
                line = line.replace("TEST_JSON_FILE", curr_chunk_file_path)
            config_file.write(line)
        config_file.close()
        print("Config file: {}".format(config_path_to_write_in_script))
        
        # Scripts.
        eval_script_path = os.path.join(
            output_scripts_folder,
            "ego4d_scod_testset_{}_{}_{}.sh".format(task_type, output_scripts_folder_name, i+1)
        )
        eval_script_path_to_run = eval_script_path.replace("{}/".format(glip_code_base_folder), "")
        eval_script = open(eval_script_path, "w")
        for line in eval_script_lines:
            if "MODEL_CONFIG" in line:
                line = line.replace("TODO", model_config)
            elif "MODEL_WEIGHT" in line:
                line = line.replace("TODO", model_checkpoint)
            elif "TASK_CONFIG" in line:
                line = line.replace("TODO", config_path_to_write_in_script)
            elif "OUTPUT_DIR" in line:
                line = line.replace("TODO", os.path.join(output_eval_results_folder, "chunk_{}".format(i+1)))
            elif "CUDA_VISIBLE_DEVICES" in line:
                line = line.replace("TODO", str(gpu_id))
            eval_script.write(line)
        eval_script.close()
        print("sh {}".format(eval_script_path_to_run))
        
        print("-"*50)
        
    assert per_chunk_images == len(input_data["images"])

    return None

In [167]:
import copy

# Params.
input_file_to_chunk = (
    # "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    # "/paper_data/all_frames/narrated/full_sentence/val_scod_all_frames_narrated_full_sentence.json"
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/test_set/blip_gpt_v3/test_res_BLIP.json"
)
num_chunks = 4
task_type = "narrated"  # "od"
model_checkpoint = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    # "/paper_models/all_frames/narrated/full_sentence/ego4d_scod_all_frames_narrated_full_sentence"
    # "/paper_models/all_frames/narrated/gpt_v1_srl_arg1_with_tool"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool"
    # "/ft_task_1/model_0045000.pth"
    "/paper_models/all_frames/narrated/gpt_v3_srl_arg1_with_tool"
    "/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr"
    "/ft_task_1/model_0045000.pth"
)
model_config = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/baselines/GLIP/glip_model_zoo/glip_Swin_L.yaml"
)
# output_scripts_folder_name = "full_sentence"
# output_scripts_folder_name = "gpt_v1_srl_arg1_with_tool"
output_scripts_folder_name = "gpt_v3_srl_arg1_with_tool"

assert "ft_task_1" in model_checkpoint
output_eval_results_folder = os.path.join(
    model_checkpoint.replace("ft_task_1", "XXX").split("XXX")[0],
    "eval_scod_testset"
)

glip_code_base_folder = "../../GLIP"


##################################################################################################
_ = chunk_ego4d_testset(
    input_file_to_chunk=input_file_to_chunk,
    num_chunks=num_chunks,
    glip_code_base_folder=glip_code_base_folder,
    task_type=task_type,
    output_scripts_folder_name=output_scripts_folder_name,
    output_scripts_folder_root="paper_scripts/ego4d_scod/test_set",
    output_configs_folder_root="configs/paper_results/ego4d_scod/test_set",
    output_chunked_file_root="/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/test_set",
    output_eval_results_folder=output_eval_results_folder,
    model_checkpoint=model_checkpoint,
    model_config=model_config,
)

--------------------------------------------------
GPU: 0
File: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/test_set/gpt_v3_srl_arg1_with_tool/test_res_BLIP_1.json
Data Points: 165724
Config file: configs/paper_results/ego4d_scod/test_set/gpt_v3_srl_arg1_with_tool/ego4d_scod_testset_narrated_gpt_v3_srl_arg1_with_tool_1.yaml
sh paper_scripts/ego4d_scod/test_set/gpt_v3_srl_arg1_with_tool/ego4d_scod_testset_narrated_gpt_v3_srl_arg1_with_tool_1.sh
--------------------------------------------------
GPU: 1
File: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/test_set/gpt_v3_srl_arg1_with_tool/test_res_BLIP_2.json
Data Points: 165724
Config file: configs/paper_results/ego4d_scod/test_set/gpt_v3_srl_arg1_with_tool/ego4d_scod_testset_narrated_gpt_v3_srl_arg1_with_tool_2.yaml
sh paper_scripts/ego4d_scod/test_set/gpt_v3_srl_arg1_with_tool/ego4d_scod_testset_narrated_gpt_v3_srl_arg1_with_tool_2.sh
----------------------------------------------

### Merge to `One` File

**TODO: Filter out tools, only record OOC!**

In [181]:
model_checkpoint = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge"
    # "/paper_models/all_frames/narrated/full_sentence/ego4d_scod_all_frames_narrated_full_sentence"
    # "/paper_models/all_frames/narrated/gpt_v1_srl_arg1_with_tool"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool"
    # "/ft_task_1/model_0045000.pth"
    "/paper_models/all_frames/narrated/gpt_v3_srl_arg1_with_tool"
    "/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr"
    "/ft_task_1/model_0045000.pth"
)

assert "ft_task_1" in model_checkpoint
output_eval_results_folder = os.path.join(
    model_checkpoint.replace("ft_task_1", "XXX").split("XXX")[0],
    "eval_scod_testset"
)

In [182]:
model_iter = model_checkpoint.split("/")[-1].split(".")[0]
model_iter_k = int(int(model_iter.split("_")[-1]) / 1000)

inference_folder = os.path.join(
    output_eval_results_folder,
    "eval_at_{}k".format(model_iter_k),
    "inference/narrated_ego4d_test",
)
if not os.path.exists(inference_folder):
    os.makedirs(inference_folder)

all_results = []
for i in range(num_chunks):
    actual_results_folder = os.path.join(
        output_eval_results_folder,
        "chunk_{}".format(i+1),
        "eval",
    )
    actual_results_json = os.path.join(
        actual_results_folder,
        "{}/inference/narrated_ego4d_test/bbox.json".format(model_iter)
    )
    print("Adding: {}".format(actual_results_json))
    assert os.path.exists(actual_results_json)
    all_results += json.load(open(actual_results_json))
    
json.dump(
    all_results,
    open(os.path.join(inference_folder, "bbox.json"), "w")
)
print("Saving file to: {}".format(os.path.join(inference_folder, "bbox.json")))

Adding: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated/gpt_v3_srl_arg1_with_tool/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr/eval_scod_testset/chunk_1/eval/model_0045000/inference/narrated_ego4d_test/bbox.json
Adding: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated/gpt_v3_srl_arg1_with_tool/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr/eval_scod_testset/chunk_2/eval/model_0045000/inference/narrated_ego4d_test/bbox.json
Adding: /local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated/gpt_v3_srl_arg1_with_tool/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_new_condstr/eval_scod_testset/chunk_3/eval/model_0045000/inference/narrated_ego4d_test/bbox.json
Adding: /local1/telinwu/research/resources/Ego

# END